In [17]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set paths for training and validation
TRAIN_PATH = '../DeepGlobe Dataset/train_subSub'
VALID_PATH = '../DeepGlobe Dataset/valid_subSub'

# Parameters
PATCH_SIZE = 64  # Size of the patches
IMG_SIZE = (256, 256)  # Resized image size for input to the network
BATCH_SIZE = 16
EPOCHS = 50

# Function to load images and their corresponding masks
def load_images_from_directory(directory, image_size=IMG_SIZE):
    images, masks = [], []
    for filename in tqdm(os.listdir(directory)):
        if 'sat' in filename:
            # Load satellite image
            sat_img = load_img(os.path.join(directory, filename), target_size=image_size)
            sat_img = img_to_array(sat_img) / 255.0  # Normalize
            images.append(sat_img)
            
            # Load corresponding mask image
            mask_filename = filename.replace('sat.jpg', 'mask.png')
            mask_img = load_img(os.path.join(directory, mask_filename), target_size=image_size, color_mode="grayscale")
            mask_img = img_to_array(mask_img) / 255.0  # Normalize to 0-1
            masks.append(mask_img)
    return np.array(images), np.array(masks)

# Function to create patches from images and masks
def create_patches(images, masks, patch_size=PATCH_SIZE):
    img_patches, mask_patches = [], []
    for img, mask in zip(images, masks):
        for i in range(0, IMG_SIZE[0], patch_size):
            for j in range(0, IMG_SIZE[1], patch_size):
                img_patch = img[i:i+patch_size, j:j+patch_size]
                mask_patch = mask[i:i+patch_size, j:j+patch_size]
                img_patches.append(img_patch)
                mask_patches.append(mask_patch)
    return np.array(img_patches), np.array(mask_patches)

# Load the training data
train_images, train_masks = load_images_from_directory(TRAIN_PATH)

# Split into patches
train_img_patches, train_mask_patches = create_patches(train_images, train_masks)

# Build a traditional CNN for image segmentation
def build_cnn(input_shape=(PATCH_SIZE, PATCH_SIZE, 3)):
    model = Sequential()
    
    # Add convolutional layers
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add a fully connected layer for classification
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    
    # Output layer reshaped to match the mask size (patch_size x patch_size x 1)
    model.add(Dense(PATCH_SIZE * PATCH_SIZE, activation='sigmoid'))
    model.add(Reshape((PATCH_SIZE, PATCH_SIZE, 1)))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Build the model
model = build_cnn(input_shape=(PATCH_SIZE, PATCH_SIZE, 3))
model.summary()

# Train the model
model.fit(train_img_patches, train_mask_patches, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

# Function to predict masks for the validation set
def predict_and_save_masks(model, valid_dir, output_dir, image_size=IMG_SIZE, patch_size=PATCH_SIZE):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(valid_dir):
        if 'sat' in filename:
            # Load the validation satellite image
            sat_img = load_img(os.path.join(valid_dir, filename), target_size=image_size)
            sat_img = img_to_array(sat_img) / 255.0  # Normalize
            sat_patches = create_patches([sat_img], [sat_img])[0]
            
            # Predict mask for each patch
            pred_patches = model.predict(sat_patches)
            
            # Reconstruct full-size mask from patches
            pred_mask = np.zeros(image_size, dtype=np.float32)
            patch_index = 0
            for i in range(0, image_size[0], patch_size):
                for j in range(0, image_size[1], patch_size):
                    pred_mask[i:i+patch_size, j:j+patch_size] = pred_patches[patch_index].reshape(patch_size, patch_size)
                    patch_index += 1
            
            # Save predicted mask
            output_filename = filename.replace('sat.jpg', 'predicted_mask.png')
            cv2.imwrite(os.path.join(output_dir, output_filename), (pred_mask * 255).astype(np.uint8))

# Predict and save masks for validation images
VALID_OUTPUT_DIR = 'predicted_masks'
predict_and_save_masks(model, VALID_PATH, VALID_OUTPUT_DIR)

100%|██████████| 510/510 [00:05<00:00, 87.17it/s] 
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4096)           │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 64, 64, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,517,184 (36.31 MB)

 Trainable params: 9,517,184 (36.31 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 18s 74ms/step - accuracy: 0.9252 - loss: 0.2473 - val_accuracy: 0.9431 - val_loss: 0.2158
Epoch 2/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.9566 - loss: 0.1703 - val_accuracy: 0.9431 - val_loss: 0.2134
Epoch 3/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.9570 - loss: 0.1674 - val_accuracy: 0.9431 - val_loss: 0.2102
Epoch 4/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 18s 80ms/step - accuracy: 0.9569 - loss: 0.1689 - val_accuracy: 0.9431 - val_loss: 0.2107
Epoch 5/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.9581 - loss: 0.1626 - val_accuracy: 0.9431 - val_loss: 0.2133
Epoch 6/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 19s 85ms/step - accuracy: 0.9585 - loss: 0.1608 - val_accuracy: 0.9431 - val_loss: 0.2087
Epoch 7/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 16s 68ms/step - accuracy: 0.9568 - loss: 0.1638 - val_accuracy: 0.9431 - val_loss: 0.2086
Epoch 8/50
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.9568 - loss: 0.1599 - 

KeyboardInterrupt: 